## Introduction

## Importing Necessary Libraries

In [90]:
!pip install matplotlib

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import Image

from math import *
from dataclasses import dataclass, asdict
from enum import Enum
from typing import List
import random
import pandas as pd


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


## Defining Constants and Variables

In [134]:
class Basis(Enum):
    Rectilinear = ("H", "V")
    Diagonal = ("D", "A")

class States(Enum):
    H = (1, 0)
    V = (0, 1)
    D = (1/1/sqrt(2), 1/1/sqrt(2))
    A = (1/1/sqrt(2), -1/1/sqrt(2))
    
    @property
    def basis(self):
        return Basis.Rectilinear if self.name in Basis.Rectilinear.value else Basis.Diagonal
    
    @property
    def bit(self):
        return 0 if self.name in ("H", "D") else 1

@dataclass
class Measurement:
    d1h: bool
    d1v: bool
    d2h: bool
    d2v: bool
    
@dataclass
class SimulationResults:
    alice_states: List[States]
    bob_states: List[States]
    measurements: List[Measurement]
    valid_bits: List[bool]

## Defining Helper Functions

#### Expected Photon Count

In [128]:
N1v = lambda ah, av, bh, bv: (av - bv)**2 / 2
N1h = lambda ah, av, bh, bv: (ah - bh)**2 / 2
N2v = lambda ah, av, bh, bv: (av + bv)**2 / 2
N2h = lambda ah, av, bh, bv: (ah + bh)**2 / 2

#### Simulation

In [129]:
def simulate_measurement(alice: States, bob: States) -> Measurement:
    
    detector_weights = [
        N1v(*alice.value, *bob.value),
        N1h(*alice.value, *bob.value),
        N2v(*alice.value, *bob.value),
        N2h(*alice.value, *bob.value),
    ]
    
    detector_hits = random.choices(["d1v", "d1h", "d2v", "d2h"], weights=detector_weights, k=2)
    
    d1v = "d1v" in detector_hits
    d1h = "d1h" in detector_hits
    d2v = "d2v" in detector_hits
    d2h = "d2h" in detector_hits
    
    return Measurement(d1h=d1h, d1v=d1v, d2h=d2h, d2v=d2v)


def simulate_experiment(steps: int) -> SimulationResults:
    
    alice_states = []
    bob_states = []
    measurements = []
    valid_bits = []
    
    for step in range(steps):
        
        alice = random.choice(list(States))
        bob = random.choice(list(States))
        measurement = simulate_measurement(alice, bob)
        is_valid = alice.basis == bob.basis
        
        alice_states.append(alice)
        bob_states.append(bob)
        measurements.append(measurement)
        valid_bits.append(is_valid)
        
    return SimulationResults(alice_states=alice_states, bob_states=bob_states, measurements=measurements, valid_bits=valid_bits)

In [141]:
results = simulate_experiment(100000)

df = pd.DataFrame(asdict(results))

df[["d1h", "d1v", "d2h", "d2v"]] = pd.json_normalize(df["measurements"])

df["b_plus"] = (df["d1h"] & df["d2v"]) | (df["d1v"] & df["d2h"])
df["b_minus"] = (df["d1h"] & df["d2h"]) | (df["d1v"] & df["d2v"])

df[df["valid_bits"]]

TypeError: choices() got an unexpected keyword argument 'replace'

### Testing

In [121]:
for alice in States:
    for bob in States:
        
        same_basis = alice.basis == bob.basis
        
        print(f"{alice.name}, {bob.name}: {same_basis}")

H, H: True
H, V: True
H, D: False
H, A: False
V, H: True
V, V: True
V, D: False
V, A: False
D, H: False
D, V: False
D, D: True
D, A: True
A, H: False
A, V: False
A, D: True
A, A: True


In [ ]:
steps = 10000


outputs = {
    "Alice": [],
    "Bob": [],
    "d1v": [],
    "d1h": [],
    "d2v": [],
    "d2h": []
}

for alice in States:
    for bob in States:
        
        d1 = 0
        d2 = 0
        d3 = 0
        d4 = 0
        
        for i in range(steps):
            detector_weights = [
                N1v(*alice.value, *bob.value),
                N1h(*alice.value, *bob.value),
                N2v(*alice.value, *bob.value),
                N2h(*alice.value, *bob.value),
            ]
    
            detector_hits = random.choices(["d1v", "d1h", "d2v", "d2h"], weights=detector_weights, k=2)
            d1 += detector_hits.count("d1v")
            d2 += detector_hits.count("d1h")
            d3 += detector_hits.count("d2v")
            d4 += detector_hits.count("d2h")

        
        outputs["Alice"].append(alice)
        outputs["Bob"].append(bob)


        outputs['d1v'].append(round(d1/steps, 2))
        outputs['d1h'].append(round(d2/steps, 2))
        outputs['d2v'].append(round(d3/steps, 2))
        outputs['d2h'].append(round(d4/steps, 2))
        
df = pd.DataFrame(outputs)  
df = df.set_index(["Alice", "Bob"])  

df

d1v   d1h   d2v   d2h
Alice    Bob                             
States.H States.H  0.00  0.00  0.00  2.00
         States.V  0.50  0.50  0.48  0.51
         States.D  0.25  0.04  0.25  1.46
         States.A  0.25  0.04  0.26  1.45
States.V States.H  0.49  0.50  0.50  0.51
         States.V  0.00  0.00  2.00  0.00
         States.D  0.04  0.25  1.45  0.26
         States.A  1.46  0.25  0.04  0.25
States.D States.H  0.24  0.04  0.26  1.46
         States.V  0.04  0.25  1.46  0.25
         States.D  0.00  0.00  1.00  1.00
         States.A  1.01  0.00  0.00  0.99
States.A States.H  0.25  0.04  0.26  1.44
         States.V  1.45  0.25  0.04  0.26
         States.D  1.00  0.00  0.00  1.00
         States.A  0.00  0.00  1.00  1.00